## 데이터 준비와 전처리

In [5]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [7]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(15)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


## 분석해 봅시다.

In [9]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [10]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
# movies와 ratings를 합쳐준다.
data= pd.merge(movies, ratings)

In [12]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = [ 'user_id','title', 'rating']
data = data[using_cols]
data.head(10)

,user_id,title,rating
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [13]:
data['title'] = data['title'].str.lower() 

In [16]:
#가장 인기있는 영화 30개(인기순)
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [17]:
# 유저별 몇 편의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [18]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['men in black (1997)' , 'fugitive, the (1993)' ,'terminator, the (1984)' ,
               'back to the future (1985)' ,'forrest gump (1994)']

# 'kimin'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kimin']*5, 'title': my_favorite, 'rating':[5]*5})

# user_id에 'kimin'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
if not data.isin({'user_id':['kimin']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,rating
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


## 모델에 활용하기 위한 전처리

In [19]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [22]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kimin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['forrest gump (1994)'])

6039
342


In [23]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,rating
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1419,5
1,6039,439,5
2,6039,1122,5
3,6039,1152,5


## CSR matrix를 직접 만들어 봅시다.

In [24]:
data['rating']

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64

In [25]:
print(data.rating)

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64


In [26]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.rating, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [27]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [28]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [29]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [30]:
als_model.fit(csr_data_transpose)

In [31]:
kimin, back_to_the_future = user_to_idx['kimin'], title_to_idx['back to the future (1985)']
kimin_vector, back_to_the_future_vector = als_model.user_factors[kimin], als_model.item_factors[back_to_the_future]

In [32]:
kimin_vector

array([ 0.01211003,  0.03890939,  0.08449454, -0.8152703 , -0.37206334,
       -0.7583493 ,  0.22418347, -1.0113535 , -0.87673783, -0.06011937,
        1.0947958 , -0.17893732,  0.08240743,  0.64854014, -0.51209253,
        0.6901732 ,  0.01151973,  0.4272302 ,  0.1749072 , -0.53611994,
       -0.2816594 , -0.02802304, -0.11736549, -0.04342868,  0.15207136,
       -0.15957841,  0.7740451 , -0.4724301 ,  0.26079747,  0.35419685,
        0.37893856, -0.411494  ,  1.0924221 , -0.7273062 ,  0.7299455 ,
        0.71575093, -0.10807151, -0.36924842,  0.48238423,  0.05341285,
        0.00566364,  0.80692375, -0.3420262 ,  0.31476676,  0.15170176,
       -0.00922048, -0.10423599,  0.30047536, -0.4374511 , -0.39949417,
       -0.22832984,  0.18571113, -0.6400033 , -0.60119456, -0.11848782,
        0.35173318,  0.42603093,  0.03993591,  0.4633191 , -0.10541896,
        0.01525095, -0.16488357,  0.22104298, -0.32947642,  0.2038296 ,
       -0.51842344,  0.26933044,  0.01037226,  0.06899752, -0.01

In [33]:
back_to_the_future_vector

array([-3.55391251e-03,  2.87794936e-02, -9.15153790e-03, -5.94950689e-04,
       -1.87447518e-02, -8.98510590e-03,  4.17002365e-02, -1.92144103e-02,
       -3.47485058e-02, -2.47050170e-02,  2.25903243e-02, -8.09754338e-03,
        1.38907405e-02,  7.65880663e-03,  1.32850995e-02,  2.79524438e-02,
       -1.43684791e-02,  1.73857212e-02,  5.46103194e-02,  9.68917366e-03,
        1.32879084e-02,  2.62968522e-02,  5.52740134e-02, -8.08137818e-04,
        2.21185703e-02, -1.07524889e-02,  3.82199213e-02, -1.96780432e-02,
        2.19168179e-02,  3.56295332e-02,  9.07468982e-03, -6.86314702e-03,
        4.59577218e-02, -1.42222298e-02,  2.01385990e-02, -1.99414194e-02,
        2.40408108e-02, -6.94172166e-04,  3.20826024e-02, -2.15232801e-02,
        1.96125954e-02,  1.72623806e-02, -1.26206065e-02,  3.58625613e-02,
       -1.52397994e-02,  3.74546228e-03,  2.71779839e-02,  1.45452062e-03,
        1.05687301e-03,  6.58694888e-03, -2.14940030e-02,  1.87208422e-03,
       -3.29487547e-02, -

## 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [34]:
np.dot(kimin_vector, back_to_the_future_vector)

0.5726157

In [35]:
toy = title_to_idx['toy story (1995)']
toy_vector = als_model.item_factors[toy]
np.dot(kimin_vector, toy_vector)

0.20457956

## 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [36]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [37]:
get_similar_title('men in black (1997)') 

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'fifth element, the (1997)',
 'matrix, the (1999)',
 'lost world: jurassic park, the (1997)',
 'face/off (1997)',
 'rocky horror picture show, the (1975)']

In [38]:
get_similar_title('toy story (1995)') 

['toy story (1995)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'babe (1995)',
 'aladdin (1992)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 "there's something about mary (1998)"]

In [39]:
get_similar_title('back to the future (1985)') 

['back to the future (1985)',
 "ferris bueller's day off (1986)",
 'back to the future part ii (1989)',
 'ghostbusters (1984)',
 'when harry met sally... (1989)',
 'big (1988)',
 'fish called wanda, a (1988)',
 'e.t. the extra-terrestrial (1982)',
 'cocoon (1985)',
 'bull durham (1988)']

## 내가 가장 좋아할 만한 영화들을 추천받기.

In [40]:
user = user_to_idx['kimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

terminator 2: judgment day (1991) 0000.646
matrix, the (1999) 0000.569
jurassic park (1993) 0000.533
braveheart (1995) 0000.401
total recall (1990) 0000.390
star wars: episode vi - return of the jedi (1983) 0000.371
sixth sense, the (1999) 0000.341
star wars: episode v - the empire strikes back (1980) 0000.340
hunt for red october, the (1990) 0000.320
groundhog day (1993) 0000.319
star wars: episode iv - a new hope (1977) 0000.307
die hard (1988) 0000.296
speed (1994) 0000.262
aliens (1986) 0000.253
independence day (id4) (1996) 0000.246
e.t. the extra-terrestrial (1982) 0000.227
face/off (1997) 0000.226
saving private ryan (1998) 0000.225
raiders of the lost ark (1981) 0000.214
ghostbusters (1984) 0000.213


In [41]:
rihanna = title_to_idx['die hard (1988)']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('terminator, the (1984)', 0.2095254664143465),
 ('fugitive, the (1993)', 0.16352848258195082),
 ('back to the future (1985)', 0.021186287839777312),
 ('forrest gump (1994)', -0.030889536059376846),
 ('men in black (1997)', -0.06954584609152567)]

## 마무리

이번 프로젝트는 추천 시스템에 대한 내용이었다.  
사용자가 선호하는 항목을 토대로 유사한 항목을 추천하는 시스템이다.  
추천 시스템의 기술로는 협업 필터링, 콘텐츠 기반 필터링이 있다.  
여기서는 비슷한 영화를 추천하거나 도출하는 활동만 있는 것으로 봐서 콘텐츠 기반 필터링만 있는 듯 하다.  
여기서는 특이하게 csv파일을 두개 받았다.  
내게 필요한 칼럼은 양쪽 모두에 있어서 하나만 쓸 수 없었다.  
어떻게 사용할까 고민하다가 전에 배운 merge를 사용했다.  
모델을 성공적으로 돌리니까 추천 시스템이 정상적으로 동작했다.  
iterations를 높일 수록 내가 선택한 영화와 행렬곱을 한 결과가 높아졌다.  
높은 정확도가 나오면 좋을 것 같지만, 내가 선호하는 다른 영화의 정확도는 떨어지는 단점이 있다.  
이 두가지를 적절하게 조절하는 시스템이 좋은 추천 시스템인듯 하다. 